In [1]:
import numpy
import random
from matplotlib import pyplot
from matplotlib import colors
from matplotlib import font_manager
import matplotlib.dates as mdates
import matplotlib.ticker as mticker
import matplotlib.patches as mpatches
import matplotlib.lines as mlines
import datetime
import cartopy.crs
from cartopy.mpl.gridliner import LATITUDE_FORMATTER, LONGITUDE_FORMATTER
from PIL import Image
import pandas
from sklearn.linear_model import LinearRegression, LogisticRegression
from mpl_toolkits.mplot3d import Axes3D
import seaborn
import scipy
from scipy import stats
from scipy.stats import poisson, ttest_ind
from scipy import linalg
import xarray as xr
import netCDF4
import cftime
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Define Directory
Diri = "data/orig/"

#
# Define Variables of Research
# Near Surface Air Temperature
Var1 = "tas"
Temp_Files = os.listdir(Diri+Var1+"/global/")
# Net Radiative Flux at Top of Atmosphere
Var2 = "netTOA"
Net_TOA_Files = os.listdir(Diri+Var2+"/global/")
# Precipitation
Var3 = "pr"
Precip_Files = os.listdir(Diri+Var3+"/global/")

In [3]:
# Open Temperature Files
for File in Temp_Files:
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    Time = Parts[3].split(".")[0]
    if Time=="150":
        Temp_Files.remove(File)

In [4]:
# List and Open Net Top of Atmosphere Radiation Files
for File in Net_TOA_Files: 
#    print(File)
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    Time = Parts[3].split(".")[0]
#    print(Time)
    if Time=="150":
#        print("Remove model")
        Net_TOA_Files.remove(File)

In [5]:
# Open Precipitation Files
for File in Precip_Files:
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    Time = Parts[3].split(".")[0]
    if Time=="150":
        Precip_Files.remove(File)

In [6]:
# Create Pertubations Library
Perturbations = {} 
for File in Temp_Files: 
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    if Experiment!="control": 
        if Model in list(Perturbations):
            Perturbations[Model].append(Experiment)
        else:
            Perturbations[Model] = [Experiment]
print (Perturbations)

{'MPIESM12': ['abrupt4x'], 'GISSE2R': ['abrupt4x'], 'IPSLCM5A': ['abrupt4x'], 'CCSM3': ['abrupt4x'], 'HadGEM2': ['abrupt4x'], 'CESM104': ['abrupt4x'], 'CCSM3II': ['abrupt1400ppm'], 'ECHAM5MPIOM': ['abrupt4x'], 'FAMOUS': ['abrupt4x'], 'CNRMCM61': ['abrupt4x'], 'HadCM3L': ['abrupt4x'], 'MPIESM11': ['abrupt4x']}


In [7]:
# Create Abrupt 4x Pertubations Library
Pert_4x = {}
for File in Temp_Files: 
    Parts = File.split("_")
    Model = Parts[1]
    Experiment = Parts[2]
    if Experiment!="control": 
        if Experiment=="abrupt4x" or Experiment=="abrupt1400ppm":
            Pert_4x[Model] = Experiment
print (Pert_4x)
#
# Define Number of Datasets
Num_Datasets = len(Pert_4x)
print (Num_Datasets)

{'MPIESM12': 'abrupt4x', 'GISSE2R': 'abrupt4x', 'IPSLCM5A': 'abrupt4x', 'CCSM3': 'abrupt4x', 'HadGEM2': 'abrupt4x', 'CESM104': 'abrupt4x', 'CCSM3II': 'abrupt1400ppm', 'ECHAM5MPIOM': 'abrupt4x', 'FAMOUS': 'abrupt4x', 'CNRMCM61': 'abrupt4x', 'HadCM3L': 'abrupt4x', 'MPIESM11': 'abrupt4x'}
12


In [8]:
# Create List of Models
#Model_List = []
#for Model in list(Pert_4x):
#    Model_List.append(Model)
Model_List = ['MPIESM12', 'MPIESM11', 'IPSLCM5A', 'HadGEM2', 'HadCM3L', 'GISSE2R', 'FAMOUS', 'ECHAM5MPIOM', \
'CNRMCM61', 'CESM104', 'CCSM3II', 'CCSM3']
print (Model_List)

['MPIESM12', 'MPIESM11', 'IPSLCM5A', 'HadGEM2', 'HadCM3L', 'GISSE2R', 'FAMOUS', 'ECHAM5MPIOM', 'CNRMCM61', 'CESM104', 'CCSM3II', 'CCSM3']


In [9]:
# Create Function to Open Temperature Files
def Open_Temp_Files(Temp_Files, Model):
# Open Temperature Control Files
    Temp_Control_Filename = [i for i in Temp_Files if i.startswith("tas_"+Model+"_"+"control"+"_")][0]
    Temp_Control_Data = xr.open_dataset(Diri+"tas/global/"+Temp_Control_Filename)
    Temp_Control = Temp_Control_Data.tas
    Years_Control = range(len(Temp_Control))
#
# Open Temperature 4x Files
    Temp_4x_Filename = [i for i in Temp_Files if i.startswith("tas_"+Model+"_"+Pert_4x[Model]+"_")][0]
    Temp_4x_Data = xr.open_dataset(Diri+"tas/global/"+Temp_4x_Filename)
    Temp_4x = Temp_4x_Data.tas
    Years_4x = range(len(Temp_4x))
#
# Return Results
    return (Temp_Control, Years_Control, Temp_4x, Years_4x)

In [10]:
# Create Function to Create Smoothen Temperature 4x and Control Lines
def Temp_Smooth(Temp_Control, Years_Control, Temp_4x, Years_4x, Model):
### Take the difference of each time step to the time-averaged control simulation imbalance
    Temp_Control_Smooth = Temp_Control.mean() + Temp_4x*0
#
# Return Results
    return (Temp_Control_Smooth)

In [11]:
# Create Function to Open Net TOA Files
def Open_Net_TOA_Files(Net_TOA_Files, Model):
# Open Net TOA Control Files
    Net_TOA_Control_Filename = [i for i in Net_TOA_Files if i.startswith("netTOA_"+Model+"_"+"control"+"_")][0]
    Net_TOA_Control_Data = xr.open_dataset(Diri+"netTOA/global/"+Net_TOA_Control_Filename)
    Net_TOA_Control_Data["netTOA"][Net_TOA_Control_Data["netTOA"] < 0] = numpy.nan
    Net_TOA_Control = Net_TOA_Control_Data.netTOA
    Net_TOA_Control[Net_TOA_Control < 0] = numpy.nan
    Years_Control = range(len(Net_TOA_Control))
    Years_Control = numpy.array(Years_Control)
#
# Open Net TOA 4x Files
    Net_TOA_4x_Filename = [i for i in Net_TOA_Files if i.startswith("netTOA_"+Model+"_"+Pert_4x[Model]+"_")][0]
    Net_TOA_4x_Data = xr.open_dataset(Diri+"netTOA/global/"+Net_TOA_4x_Filename)
    Net_TOA_4x = Net_TOA_4x_Data.netTOA
    Years_4x = range(len(Net_TOA_4x))
#
# Return Results
    return (Net_TOA_Control, Years_Control, Net_TOA_4x, Years_4x)

In [12]:
# Create Function to Create Smoothen Net TOA 4x and Control Lines
def Net_TOA_Smooth(Net_TOA_Control, Years_Control, Net_TOA_4x, Years_4x, Model):
### Atmospheric variable anomal - removing imbalances in the control and drift
### (Rugenstein et al 2019, BAMS - section 3a, fig 3)
### Take the difference of each time step to the time-averaged control simulation imbalance
    Net_TOA_Control_Smooth = Net_TOA_Control.mean() + Net_TOA_4x*0
#
# Return Results
    return (Net_TOA_Control_Smooth)

In [13]:
# Create Function to Open Precipitation Files
def Open_Precip_Files(Precip_Files, Model):
# Open Precipitation Control Files
    Precip_Control_Filename = [i for i in Precip_Files if i.startswith("pr_"+Model+"_"+"control"+"_")][0]
    Precip_Control_Data = xr.open_dataset(Diri+"pr/global/"+Precip_Control_Filename)
    Precip_Control_Data["pr"][Precip_Control_Data["pr"] < 0] = numpy.nan
    Precip_Control = Precip_Control_Data.pr
    Precip_Control[Precip_Control < 0] = numpy.nan
    Years_Control = range(len(Precip_Control))
#
# Open Precipitation 4x Files
    Precip_4x_Filename = [i for i in Precip_Files if i.startswith("pr_"+Model+"_"+Pert_4x[Model]+"_")][0]
    Precip_4x_Data = xr.open_dataset(Diri+"pr/global/"+Precip_4x_Filename)
    Precip_4x = Precip_4x_Data.pr
    Years_4x = range(len(Precip_4x))
#
# Return Results
    return (Precip_Control, Years_Control, Precip_4x, Years_4x)

In [14]:
# Create Function to Create Smoothen Precipitation 4x and Control Lines
L_Vapour = 2.5 * 10**6
def Precip_Smooth(Precip_Control, Years_Control, Precip_4x, Years_4x, Model):
### Atmospheric variable anomal - removing imbalances in the control and drift
### (Rugenstein et al 2019, BAMS - section 3a, fig 3)
### Take the difference of each time step to the time-averaged control simulation imbalance
    Precip_Control_Smooth = Precip_Control.mean() + Precip_4x*0
#
# Return Results
    return (Precip_Control_Smooth)

In [15]:
def AutocorLone(x):
    a = numpy.ma.masked_invalid(x[:-1])
    b = numpy.ma.masked_invalid(x[1:])
    msk = (~a.mask & ~b.mask)
    ac = numpy.ma.corrcoef(a[msk],b[msk])[0,1]
    #print(ac)
    return ac

In [16]:
def Detrend(x):
    xc = x - x.mean()
    t = numpy.arange(len(x))
    tc = t - t.mean()
    slope, intercept, r_value, p_value, std_err = stats.linregress(tc,xc)
### Maybe I need to include the intercept here if it's nonzero? 
    xdc = xc - slope*tc
    return xdc

In [17]:
def Corr_IgnoreNaN(x,y):
    a = numpy.ma.masked_invalid(x)
    b = numpy.ma.masked_invalid(y)
    msk = (~a.mask & ~b.mask)
    c = numpy.ma.corrcoef(a[msk],b[msk])[0,1]
    #print(ac)
    return c

In [18]:
def CalcParams_Fixed_iv(xdat,sxi,ydat,syi,pit):
    """
    Calculate the parameters, both intercepts as well as the slope of the regression
    :return:
    """
    # backwards compatibility with calcparams

    [sxi,syi,pit]=Backwards_Compat(sxi,syi,pit)

        
    xbar = xdat.mean()
    ybar = ydat.mean()

    xp = xdat-xbar
    yp = ydat-ybar

    sum_xp2 = numpy.sum(xp*xp)
    sum_yp2 = numpy.sum(yp*yp)
    sum_xpyp = numpy.sum(xp*yp)

    c1 = pit*sxi*syi*sum_xp2 - numpy.power(sxi,2)*sum_xpyp
    c2 = - numpy.power(syi,2)*sum_xp2 + numpy.power(sxi,2)*sum_yp2
    c3 = numpy.power(syi,2)*sum_xpyp - pit*sxi*syi*sum_yp2
    
    ## negative root
    bcalc = (-c2 - numpy.sqrt(numpy.power(c2,2)-4*c1*c3)) / 2 / c1 
    #balt = (-c2 + numpy.sqrt(numpy.power(c2,2)-4*c1*c3)) / 2 / c1 ### positive root appears to be unumpyhysical

    # now that slope is determined, calculate the y intercept
    yinter = ybar - bcalc * xbar

    # now done, so write back slope
    slope = bcalc

    # calculate x intercept
    xinter = -yinter / slope

    return numpy.array(slope), numpy.array(yinter), numpy.array(xinter), xbar, ybar

def CalcUnc_Fixed_iv(xdat,sxi,ydat,syi,xbar,ybar,b,pit):
    """
    Calculates the uncertainty of the slope and y
    """
    # backwards compatibility with calcunc
    [sxi,syi,pit]=Backwards_Compat(sxi,syi,pit)

        
    # let us first calculate the derivatives
    # dell theta / dell b (dthdb) calculation
    wi = Calc_Wi(sxi, syi, b, pit)
    sxyi = pit * sxi * syi

    ui = xdat - xbar
    vi = ydat - ybar
    
    sum1 = numpy.sum( wi**2. * (2 * b * (ui * vi * sxi**2. - ui**2. * sxyi) + (ui**2. * syi**2. - vi**2 * sxi**2.)) )
    sum2 = numpy.sum( wi**3. * (sxyi - b * sxi**2.) * (b**2. * (ui * vi * sxi**2 - ui**2 * sxyi) +
                                             b * (ui**2 * syi**2 - vi**2 * sxi**2) -
                                             (ui * vi * syi**2 - vi**2 * sxyi)) )
    
    # sum1 = 0.
    # sum2 = 0.
    # for it in range(len(xdat)):
    #     xi = xdat[it]
    #     yi = ydat[it]
    #     # sxi = xunc[it]
    #     # syi = yunc[it]
    #     # pit = p[it]
    #     # wi = calc_wi(xunc[it], yunc[it], b, pit)
    #     ui = xi - xbar
    #     vi = yi - ybar
    #     sum1 += wi**2. * (2 * b * (ui * vi * sxi**2. - ui**2. * sxyi) + (ui**2. * syi**2. - vi**2 * sxi**2.))
    #     sum2 += wi**3. * (sxyi - b * sxi**2.) * (b**2. * (ui * vi * sxi**2 - ui**2 * sxyi) +
    #                                              b * (ui**2 * syi**2 - vi**2 * sxi**2) -
    #                                              (ui * vi * syi**2 - vi**2 * sxyi))

    dthdb = sum1 + 4. * sum2

    # calculate the sum of all weights
    wksum = wi * len(xdat)

    sxj = sxi
    syj = syi
    pjt = pit
    wj = wi 
    sxyj = sxyi
    uj = xdat - xbar
    vj = ydat - ybar
            
    # now calculate sigasq and sigbsq
    sigasq = 0.
    sigbsq = 0.
    for it in range(len(xdat)):
        # calculate dell theta / dell xi and dell theta / dell yi

        kron_arr = numpy.array([kron(it,jt) for jt in range(len(xdat))])
        dthdxi = numpy.sum( wj**2. * (kron_arr - wi / wksum) * (b**2 * (vj * sxj**2 - 2 * uj * sxyj) +
                                                          2 * b * uj * syj**2 - vj * syj**2) ) 
        # correct equation! not equal to equation 21 in Mahon (1996)
        dthdyi = numpy.sum( wj**2. * (kron_arr - wi / wksum) * (b**2 * uj * sxj**2 + 2 * vj * sxyj -
                                                            2 * b * vj * sxj**2. - uj * syj**2) ) 
        # dthdxi = 0.
        # dthdyi = 0.
        # for jt in range(len(xdat)):
        #     # add to dthdxi and dthdyi
        #     dthdxi += wj**2. * (kron(it, jt) - wi / wksum) * (b**2 * (vj * sxj**2 - 2 * uj * sxyj) +
        #                                                       2 * b * uj * syj**2 - vj * syj**2)
        #     # correct equation! not equal to equation 21 in Mahon (1996)
        #     dthdyi += wj ** 2. * (kron(it, jt) - wi / wksum) * (b ** 2 * uj * sxj ** 2 + 2 * vj * sxyj -
        #                                                        2 * b * vj * sxj**2. - uj * syj ** 2)

        # now calculate dell a / dell xi and dell a / dell yi
        dadxi = -b * wi / wksum - xbar * dthdxi / dthdb
        dadyi = wi / wksum - xbar * dthdyi / dthdb

        # now finally add to sigasq and sigbsq
        sigbsq += dthdxi**2. * sxi**2. + dthdyi**2. * syi**2. + 2 * sxyi * dthdxi * dthdyi
        sigasq += dadxi**2. * sxi**2. + dadyi**2. * syi**2. + 2 * sxyi * dadxi * dadyi

    # now divide sigbsq
    sigbsq /= dthdb**2.

    yinterunc = numpy.sqrt(sigasq)
    slopeunc = numpy.sqrt(sigbsq)

    return numpy.array(slopeunc), numpy.array(yinterunc) 

def Backwards_Compat(sxi,syi,pit):
    try:
        if len(pit)>1: pit=pit[0]
    except:
        pass

    try:
        if len(syi)>1: syi=syi[0]
    except:
        pass
    
    try:
        if len(sxi)>1: sxi=sxi[0]
    except:
        pass
    return sxi,syi,pit

def CalcMswd_Fixed_iv(xdat, sxi, ydat, syi, pit, slope, yinter ):
    [sxi,syi,pit]=backwards_compat(sxi,syi,pit)
        
    wi = Calc_Wi(sxi, syi, slope, pit)
    mswd = numpy.sum( wi * ((ydat - slope*xdat - yinter)**2.)) / (len(xdat) - 2.)
    # now divide by degrees of freedom minus 2, since 2 fixed parameters
    return mswd

def Calc_Wi(sx, sy, b, p):
    return 1. / (sy**2 + b**2 * sx**2 - 2 * b * p * sx * sy)


def kron(i, j):
    # calculates Kronecker delta
    if i == j:
        return 1.
    else:
        return 0.


# example of implementation
#slope1, yinter1, xinter1, xbar1, ybar1 = calcparams_fixed_iv(xdat,xunc,ydat,yunc,p)
#slope1unc, yinter1unc = calcunc_fixed_iv(xdat,xunc,ydat,yunc,xbar1,ybar1,slope1,p)

In [19]:
# Calculate Regression Line using York Regression
def New_York_Reg(x,y,Corr,Std_x,Std_y):
    N = len(x)
# Just in case they didn't agree (e.g., MPIESM11)
    x = numpy.array(x)
    y = numpy.array(y)
    Slope, Intercept, xinter1, xbar1, ybar1 = CalcParams_Fixed_iv(x,Std_x,y,Std_y,Corr)
#
## Calculate Predictions for Plotting
    Predictions = numpy.array([[numpy.min(x), numpy.max(x)], \
    [numpy.min(x) * Slope + Intercept, numpy.max(x) * Slope + Intercept]])
#
## Calculate Error Bars
## t-statistic for N-2 dof at 84% confidence: appropriate for comparing overlap 
    Z84 = stats.t.ppf(0.84, N-2)
    Slope1unc, yInter1unc = CalcUnc_Fixed_iv(x,numpy.array(Std_x),y,numpy.array(Std_y),xbar1,ybar1,Slope,numpy.array(Corr))
    Error_Bar = Slope1unc * Z84
#
## Return Output
    return [Slope, Intercept, Error_Bar, Predictions]

## calculate T,P correlation

In [20]:
Model_List

['MPIESM12',
 'MPIESM11',
 'IPSLCM5A',
 'HadGEM2',
 'HadCM3L',
 'GISSE2R',
 'FAMOUS',
 'ECHAM5MPIOM',
 'CNRMCM61',
 'CESM104',
 'CCSM3II',
 'CCSM3']

In [21]:
# Plot Precipitation Anomaly VS Temperature Anomaly Graph For All 12 Models
Precip_1_21_151_Slopes = numpy.zeros((12,3))
Precip_1_21_151_Error_Bars = numpy.zeros((12,3))
Precip_Corr_array = numpy.zeros((12,1))

i = 0
j = -1
Count = -1
for Model in list(Model_List):
    if j < 2:
        j += 1
    else:
        j = 0
        i += 1
    Count += 1
#    Axes = Fig.add_subplot(4,3, Count)
#
# Open Temperature Control and Temperature 4x Files Using Function
    Temp_Control, Years_Control, Temp_4x, Years_4x = Open_Temp_Files(Temp_Files, Model)
#
# Create Smoothen Temperature 4x and Control Lines Using Function
    Temp_Control_Smooth = Temp_Smooth(Temp_Control, Years_Control, Temp_4x, Years_4x, Model)
#
# Calculate Temperature Anomaly
    Temp_Anomaly = Temp_4x - Temp_Control_Smooth
#
# Open Precipitation Control and Precipitation 4x Files Using Function
    Precip_Control, Years_Control, Precip_4x, Years_4x = Open_Precip_Files(Precip_Files, Model)
#
# Create Smoothen Precipitation 4x and Control Lines Using Function
    Precip_Control_Smooth = Precip_Smooth(Precip_Control, Years_Control, Precip_4x, Years_4x, Model)
#
# Calculate Precipitation Anomaly
    Precip_Control =  Precip_Control * L_Vapour
    Precip_Anomaly = (Precip_4x - Precip_Control_Smooth) * L_Vapour
    if Model=="CCSM3II":
        Precip_Anomaly = Precip_Anomaly * 1000
#
# Calculate Correlation and Standard Deviations
    PI_Len = numpy.min([len(Temp_Control),len(Precip_Control)])
    Precip_Corr = Corr_IgnoreNaN(Temp_Control[:PI_Len],Precip_Control[:PI_Len])
    Precip_Std_x = float(numpy.std(Temp_Control))
    Precip_Std_y = float(numpy.std(Precip_Control))

# Store Correlation in  Array
    Precip_Corr_array[Count][0] = Precip_Corr



In [22]:
Precip_Corr_array

array([[ 0.50074918],
       [ 0.4802978 ],
       [ 0.58541757],
       [ 0.70582386],
       [ 0.46451961],
       [-0.01958851],
       [ 0.73188028],
       [ 0.89833324],
       [ 0.828817  ],
       [ 0.71648305],
       [ 0.66198156],
       [ 0.62796933]])

## calculate T,N correlation

In [23]:
# Plot Net TOA Flux Anomaly VS Temperature Anomaly Graph For All 12 Models
Net_TOA_1_21_151_Slopes = numpy.zeros((12,3))
Net_TOA_1_21_151_Error_Bars = numpy.zeros((12,3))
Net_TOA_Corr_array = numpy.zeros((12,1))

i = 0
j = -1
Count = -1
for Model in list(Model_List):
    if j < 2:
        j += 1
    else:
        j = 0
        i += 1
    Count += 1
#    Axes = Fig.add_subplot(4,3, Count)
#
# Open Temperature Control and Temperature 4x Files Using Function
    Temp_Control, Years_Control, Temp_4x, Years_4x = Open_Temp_Files(Temp_Files, Model)
#
# Create Smoothen Temperature 4x and Control Lines Using Function
    Temp_Control_Smooth = Temp_Smooth(Temp_Control, Years_Control, Temp_4x, Years_4x, Model)
#
# Calculate Temperature Anomaly
    Temp_Anomaly = Temp_4x - Temp_Control_Smooth
#
# Open Net TOA Flux Control and Net TOA Flux 4x Files Using Function
    Net_TOA_Control, Years_Control, Net_TOA_4x, Years_4x = Open_Net_TOA_Files(Net_TOA_Files, Model)
#
# Create Smoothen Net TOA Flux 4x and Control Lines Using Function
    Net_TOA_Control_Smooth = Net_TOA_Smooth(Net_TOA_Control, Years_Control, Net_TOA_4x, Years_4x, Model)
#
# Calculate Net TOA Flux Anomaly
    Net_TOA_Anomaly = Net_TOA_4x - Net_TOA_Control_Smooth
#
# Calculate Correlation and Standard Deviations
    if Model != "CCSM3":
        PI_Len = numpy.min([len(Temp_Control),len(Net_TOA_Control)])
        Net_TOA_Corr = Corr_IgnoreNaN(Temp_Control[:PI_Len],Net_TOA_Control[:PI_Len])
        Net_TOA_Std_x = float(numpy.std(Temp_Control))
        Net_TOA_Std_y = float(numpy.std(Net_TOA_Control))


# Store Correlation in  Array
    #Precip_Corr_array[Count][0] = Precip_Corr
    Net_TOA_Corr_array[Count][0] = Net_TOA_Corr


In [24]:
Net_TOA_Corr_array

array([[-0.47561917],
       [-0.49433414],
       [-0.05183391],
       [-0.13822923],
       [-0.18821027],
       [-0.24404411],
       [-0.12120815],
       [-0.22915201],
       [-0.38550431],
       [-0.35466603],
       [-0.20753318],
       [-0.20753318]])

## print table

In [25]:
# Create DataFrame With correlations
Correlations_Compare = pandas.DataFrame({"Model": Model_List, "r(T,P)": numpy.squeeze(Precip_Corr_array.round(2)), \
"r(T,N)": numpy.squeeze(Net_TOA_Corr_array.round(2))})
Correlations_Compare

,Model,"r(T,P)","r(T,N)"
0,MPIESM12,0.50,-0.48
1,MPIESM11,0.48,-0.49
2,IPSLCM5A,0.59,-0.05
3,HadGEM2,0.71,-0.14
4,HadCM3L,0.46,-0.19
5,GISSE2R,-0.02,-0.24
6,FAMOUS,0.73,-0.12
7,ECHAM5MPIOM,0.90,-0.23
8,CNRMCM61,0.83,-0.39
9,CESM104,0.72,-0.35


In [42]:
Filename_1 = "./"+"CSV/"+"Correlations.csv"
Correlations_Compare.to_csv(Filename_1)